# Deep dive into Pandas DataFrames 

**Read the official documentation on pandas DataFrames @ https://pandas.pydata.org/pandas-docs/stable/reference/frame.html**

**`Note:`** The notion of **chaining functions/methods** in pandas is similar to python.

DataFrames are **column oriented** unlike most common databases. And, **each column** in the dataframe is a **pandas series object**. So, any operation that can be performed on a pandas series object it can be applied to a column too.

There are **two axes** for a dataframe commonly referred to as axis 0 and 1, or the **"index"** (or 'rows') axis and the **"columns"** axis respectively. Note that, when an **operation** is applied **along axis 0**, it is applied **down the column**. Likewise, operations **along axis 1** operate **across the values in the row**.

--------------------
## Import Statements
--------------------------

In [1]:
# import statements
import numpy as np
import pandas as pd

In [2]:
# view options
pd.set_option("display.max_columns", 40)
pd.set_option("display.max_rows", 10)

---------------------------
### Importing the data
------------------------

We will be exploring a dataset from a Siena College Poll in 2018. This data has rankings of United States Presidents in various attributes. These attributes are:

• Bg = Background
• Im = Imagination
• Int = Integrity
• IQ = Intelligence
• L = Luck
• WR = Willing to take risks
• AC = Ability to compromise
• EAb = Executive ability
• LA = Leadership ability
• CAb = Communication ability
• OA = Overall ability
• PL = Party leadership
• RC = Relations with Congress
• CAp = Court appointments
• HE = Handling of economy
• EAp = Executive appointments
• DA = Domestic accomplishments
• FPA = Foreign policy accomplishments
• AM = Avoid crucial mistakes
• EV = Experts’ view
• O = Overall

In [3]:
# reading from github url

# it is a good practice to define your index column when reading the data file.
# it is generally frowned upon if you don't have an index column

url = "https://github.com/mattharrison/datasets/raw/master/data/siena2018-pres.csv"
siena_2018 = pd.read_csv(url, index_col=0)

In [42]:
siena_2018.head(3)

,Seq.,President,Party,Bg,Im,Int,IQ,L,WR,AC,EAb,LA,CAb,OA,PL,RC,CAp,HE,EAp,DA,FPA,AM,EV,O
1,1,George Washington,Independent,7,7,1,10,1,6,2,2,1,11,2,18,1,1,1,1,2,2,1,2,1
2,2,John Adams,Federalist,3,13,4,4,24,14,31,21,21,13,8,28,17,4,13,15,19,13,16,10,14
3,3,Thomas Jefferson,Democratic-Republican,2,2,14,1,8,5,14,6,6,4,4,5,5,7,20,4,6,9,7,5,5


---------------------
## Casting Datatypes
-----------------------

**Note:** The first thing we should do when we load in a dataset is to check the datatypes of each column and cast each of them to more suitable datatypes. This is to save space and speed up the process.

In [13]:
siena_2018.dtypes

Seq.         object
President    object
Party        object
Bg            int64
Im            int64
              ...  
DA            int64
FPA           int64
AM            int64
EV            int64
O             int64
Length: 24, dtype: object

---------------
## Mathematical operations on DataFrames
----------------

**Similar to series objects, Math operations for DataFrames are Index Aligned.**

Aligning will take each index entry from a particular column in the left df and match it up with every entry with the same index of the same column in the right df. This is repeated for all the overlapping columns. If any of the df has duplicate index this will cause the addition operation to behave unexpectedly i.e, it will work by process of permutating the matching indexex.

In [5]:
# s1: 3 rows and 4 columns
# s2: 2 rows and 5 columns
s1 = pd.DataFrame(
    np.linspace(2, 13, 12).reshape(3, 4),
    columns=["a1", "b1", "c1", "d1"],
    index=[1, 2, 3],
)
s2 = pd.DataFrame(
    np.linspace(2, 11, 10).reshape(2, 5),
    columns=["a1", "b1", "c1", "d1", "e1"],
    index=[2, 2],
)

In [6]:
s1 + s2

,a1,b1,c1,d1,e1
1,NaN,NaN,NaN,NaN,NaN
2,8.0,10.0,12.0,14.0,NaN
2,13.0,15.0,17.0,19.0,NaN
3,NaN,NaN,NaN,NaN,NaN


As we can see, only the **overlapping rows** (2nd row) **and columns** (a1 through d1) get added together. The other values are missing. We can use the **.add method instead of "+" and define a fill value** if we wanted, similar to what we've done in case of series objects.

In [7]:
s1.add(s2, fill_value=0)

,a1,b1,c1,d1,e1
1,2.0,3.0,4.0,5.0,NaN
2,8.0,10.0,12.0,14.0,6.0
2,13.0,15.0,17.0,19.0,11.0
3,10.0,11.0,12.0,13.0,NaN


## Looping over a DataFrame (using the `for` loop)

It is generally not a good practice and is usually frowned upon if you use for loop with your pandas dataframe. This is because, pandas built-in methods are much faster than for loop due to vectorization and you are not taking advantage of it. However some times it might be useful to use for loop in datafrmes such as, when plotting visuals.

Some iterator methods that are useful while looping over a dataframe are, **.items(), .iterrows(), .itertuples()**.

- The `.items()` method: returns a tuple of **(column name, column content as Series)**

The indexes of the returned series will be the indexes of the dataframe.

In [22]:
for col_label, col_content in siena_2018.items():
    print(f"Column Name: {col_label}")
    print(f"Data contents of the column: \n{col_content}")
    break

Column Name: Seq.
Data contents of the column: 
1      1
2      2
3      3
4      4
5      5
      ..
40    41
41    42
42    43
43    44
44    45
Name: Seq., Length: 44, dtype: object


- The `.iterrows()` method: returns a tuple of **(index, row content as Series)**

The indexes of the returned series will be the associated column names.

In [28]:
for idx, row_content in siena_2018.iterrows():
    print(f"Row index: {idx}\n")
    print(f"Data contents of the row: \n\n{row_content}")
    break

Row index: 1

Data contents of the row: 

Seq.                         1
President    George Washington
Party              Independent
Bg                           7
Im                           7
                   ...        
DA                           2
FPA                          2
AM                           1
EV                           2
O                            1
Name: 1, Length: 24, dtype: object


- The `.itertuples()` method: returns the **rows as namedtuples**

In [30]:
for row in siena_2018.itertuples():
    print(row)
    break

Pandas(Index=1, _1='1', President='George Washington', Party='Independent', Bg=7, Im=7, Int=1, IQ=10, L=1, WR=6, AC=2, EAb=2, LA=1, CAb=11, OA=2, PL=18, RC=1, CAp=1, HE=1, EAp=1, DA=2, FPA=2, AM=1, EV=2, O=1)


## Aggregations

Aggregations that are applicable to a Series object are also applicable to a DataFrame. The only difference is that the aggregations can be applied across 2 axis (i.e, index and columns).

In [35]:
# let's slice out a portion from the siena_2018 df that has numerical values
# it's good practice to use .copy() with a sliced portion of a df so that operations applied
# on the sliced df doesn't affect the original df
scores = siena_2018.loc[:, "Bg":"O"].copy()

#### Multiple aggregations on a dataframe using the `.agg` method

- **Aggregate over axis 1 (apply function to each row i.e, aggregate over the columns)**

In [45]:
scores.agg(["sum", "mean"], axis=1).tail(3)

,sum,mean
42,635.0,30.238095
43,331.0,15.761905
44,833.0,39.666667


- **Aggregate over axis 0 (apply function to each column i.e, aggregate over the rows)**

In [43]:
scores.agg(["sum", "mean"], axis=0).head(3)

,Bg,Im,Int,IQ,L,WR,AC,EAb,LA,CAb,OA,PL,RC,CAp,HE,EAp,DA,FPA,AM,EV,O
sum,968.0,957.00,990.0,990.0,990.0,953.000000,968.0,978.000000,990.0,990.0,990.0,990.0,979.00,990.0,990.0,990.0,990.0,990.0,990.0,990.0,990.0
mean,22.0,21.75,22.5,22.5,22.5,21.659091,22.0,22.227273,22.5,22.5,22.5,22.5,22.25,22.5,22.5,22.5,22.5,22.5,22.5,22.5,22.5


- Different aggregations per column

In [44]:
scores.agg({"Int": ["max", "mean"], "IQ": ["min", "mean"]})

,Int,IQ
max,44.0,NaN
mean,22.5,22.5
min,NaN,1.0


#### The `.describe` returns a dataframe with summary statistics for each numeric columns

In [57]:
scores.describe()

,Bg,Im,Int,IQ,L,WR,AC,EAb,LA,CAb,OA,PL,RC,CAp,HE,EAp,DA,FPA,AM,EV,O
count,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000
mean,22.000000,21.750000,22.500000,22.500000,22.500000,21.659091,22.000000,22.227273,22.500000,22.500000,22.500000,22.500000,22.250000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000
std,12.409674,12.519984,12.845233,12.845233,12.845233,11.892822,12.409674,12.500909,12.845233,12.845233,12.845233,12.845233,12.519984,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233,12.845233
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,11.750000,11.000000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000,11.750000
50%,22.000000,21.500000,22.500000,22.500000,22.500000,22.500000,22.000000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000,22.500000
75%,32.250000,32.250000,33.250000,33.250000,33.250000,31.250000,32.250000,32.250000,33.250000,33.250000,33.250000,33.250000,33.000000,33.250000,33.250000,33.250000,33.250000,33.250000,33.250000,33.250000,33.250000
max,43.000000,43.000000,44.000000,44.000000,44.000000,41.000000,43.000000,43.000000,44.000000,44.000000,44.000000,44.000000,43.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000,44.000000


**Note:** The count row in the summary statistics has a particular meaning in pandas. It is not the count of the rows, rather it is the count of the non-missing (not na) rows.